In [2]:
import numpy as np
import pyvista as pv
from kale.algorithms import contour_banded
from kale.helpers import add_contours
from kale import theme
import xarray as xr

%config InlineBackend.figure_format = "retina"

# Load Data

In [3]:
MESH_GEOMETRY_FILE_NAME = "nankai_121728_clean.vtk"
MESH_VALUES_FILE_NAME = "2023_03_04_00_02_03.hdf"
MESH_Z_SCALE = 0.025
TIME_STEP_INTERESTING = 1000

In [4]:
ds = xr.open_dataset(MESH_VALUES_FILE_NAME, engine="netcdf4")
data = ds["loading_rate"][0, :]
mesh = pv.read(MESH_GEOMETRY_FILE_NAME)
mesh.cell_data["data"] = data
mesh.points[:, -1] *= MESH_Z_SCALE

boundary = mesh.extract_feature_edges(
    boundary_edges=True,
    non_manifold_edges=False,
    feature_edges=False,
    manifold_edges=False,
)
boundary.clear_data()

# Banded contour for a single time

In [5]:
pl = pv.Plotter()
pl.add_mesh(
    boundary, color=theme.CONTOUR_LINE_COLOR, line_width=theme.CONTOUR_LINE_WIDTH
)
box = pv.Box(mesh.bounds)
pl.add_mesh(box, color="lightgrey", culling="front")
pl.camera.zoom(1.2)

CONTOUR_LEVELS = np.linspace(-60, 60, 21)

# add_contours(plotter, source, scalars, levels, **kwargs)
add_contours(
    pl,
    mesh,
    scalars="data",
    levels=CONTOUR_LEVELS,
    cmap="RdYlBu_r",
    show_scalar_bar=True,
    scalar_bar_args=dict(title=f"", **theme.SCALAR_BAR_OPTS),
)

pl.show()

Widget(value="<iframe src='http://escalante.kitware.com:45303/index.html?ui=P_0x7f29d2110050_0&reconnect=auto'…